## Networks

1. Create a random network (with N>10.000 nodes) using the Barabasi-Albert model, and create a meaningful plot of the degree distribution!
Once the network is ready, gradually swap edges between nodes and track the change of the network topology by looking at the changes in the degree distribution!

2. Construct a network from referencing webpages! E.g. choose webpage of the Dept. of Complex Systems as the first node of the network and create new nodes and links from the "href" references! Make a visualisation of the growing network as the diameter of the network increases! (Diameter of a graph is the maximal distance between the nodes of the network. The distance between two nodes is measured by the number of steps along the shortest path connecting the node pair.

3. During reconnecting (swapping) links between nodes, usually we need a constant quantity, which does not change during the swapping of links. E.g. one can fix the degree distribution by setting a constraint at each node, how many incoming/outgoing connection it must have. Create small networks (N<50 nodes), which are randomized with swapping the links constraining the degrees of each node. By creating a visual representation of the network track the changes of swapping of the links! Show some examples, where this type of randomization does not change the topology of the network at all.

In [ ]:
import numpy as np
import networkx as nx

import seaborn as sns
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [ ]:
# Set axtick dimensions
major_size = 6
major_width = 1.2
minor_size = 3
minor_width = 1
mpl.rcParams['xtick.major.size'] = major_size
mpl.rcParams['xtick.major.width'] = major_width
mpl.rcParams['xtick.minor.size'] = minor_size
mpl.rcParams['xtick.minor.width'] = minor_width
mpl.rcParams['ytick.major.size'] = major_size
mpl.rcParams['ytick.major.width'] = major_width
mpl.rcParams['ytick.minor.size'] = minor_size
mpl.rcParams['ytick.minor.width'] = minor_width

# Seaborn style settings
sns.set_style({'axes.axisbelow': True,
               'axes.edgecolor': '.1',
               'axes.facecolor': 'white',
               'axes.grid': True,
               'axes.labelcolor': '.15',
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.spines.top': True,
               'figure.facecolor': 'white',
               'font.family': ['sans-serif'],
               'font.sans-serif': ['Arial',
                'DejaVu Sans',
                'Liberation Sans',
                'Bitstream Vera Sans',
                'sans-serif'],
               'grid.color': '.8',
               'grid.linestyle': '--',
               'image.cmap': 'rocket',
               'lines.solid_capstyle': 'round',
               'patch.edgecolor': 'w',
               'patch.force_edgecolor': True,
               'text.color': '.15',
               'xtick.bottom': True,
               'xtick.color': '.15',
               'xtick.direction': 'in',
               'xtick.top': True,
               'ytick.color': '.15',
               'ytick.direction': 'in',
               'ytick.left': True,
               'ytick.right': True})

## 1. Create a BA-graph using networkx with N > 10.000 nodes

In [ ]:
n = 1.5e5    # Number of nodes
m = 1        # Number of new edges in each step
G = nx.barabasi_albert_graph(n=n, m=m, seed=42)

In [ ]:
# Endpoints of edges
edges_list = list(G.edges)

# Degrees of nodes
degree_list = list(G.degree)

In [ ]:
# Collect only the degree values of nodes
degree_values = np.zeros(int(n))
for i, node in enumerate(degree_list):
    degree_values[i] = node[1]

### Distribution of node degrees

In [ ]:
nrows = 1
ncols = 1
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*15, nrows*8))

axislabelsize = 22
axisticksize = 18

hist, bins = np.histogram(a=degree_values, bins=50, density=True)
# Using log-log axis for a better view
axes.loglog(bins[:-1], hist,
            c='tab:red', marker='o', ms=10, lw=0,
            zorder=3)
axes.loglog(bins[:-1], hist,
            c='tab:green', lw=3, ls='--')

axes.set_xlabel('Node ranks', fontsize=axislabelsize)
axes.set_ylabel('Dist. of node ranks', fontsize=axislabelsize)
axes.tick_params(axis='both', which='major', labelsize=axisticksize)

plt.show()

### Degrees of nodes

In [ ]:
nrows = 1
ncols = 1
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*15, nrows*8))

axislabelsize = 22
axisticksize = 18
radius = 3

axes.scatter(range(int(n)), degree_values,
             s=radius**2, alpha=0.4)

# Using log y-axis for a better view
#axes.set_xscale('log')
axes.set_yscale('log')

axes.set_xlabel('Index of nodes', fontsize=axislabelsize)
axes.set_ylabel('Node ranks', fontsize=axislabelsize)
axes.tick_params(axis='both', which='major', labelsize=axisticksize)

plt.show()

## 2. Collect href links from webpages

### a) Eötvös Loránd Tudományegyetem -- Wikipedia page

For the first try, I've used the hungarian Wikipedia article on `Eötvös Loránd Tudományegyetem`

In [ ]:
import pickle

import urllib.request
from bs4 import BeautifulSoup

On Wikipedia all article references starts with simply the `/wiki/*` path. We should sort out all other references, eg. those starting with `http://` or `https://`, since those are only off-wikipedia references/citations/other junk. Also we need to sort out references starting with `#` and with the `/w/` tag and other miscellaneous hrefs, like those which contains `:`.

In [ ]:
wiki_start = 'https://hu.wikipedia.org'
hrefs_out = './hrefs/wikipedia_hrefs/'

In [ ]:
def sort_wiki_hrefs(url):
    
    html_page = urllib.request.urlopen(url)
    soup = BeautifulSoup(html_page, 'html.parser')

    hrefs = []
    for link in soup.findAll('a'):
        tmp_link = link.get('href')
        if tmp_link is not None:
            if(tmp_link[:6] == '/wiki/'
               and
               ':' not in tmp_link):
                hrefs.append(wiki_start + tmp_link)

    # Sort out references which are listed multiple times
    return list(np.unique(hrefs))

### Starting URL

In [ ]:
start_url = 'https://hu.wikipedia.org/wiki/E%C3%B6tv%C3%B6s_Lor%C3%A1nd_Tudom%C3%A1nyegyetem'

In [ ]:
hrefs_0 = sort_wiki_hrefs(url=start_url)

In [ ]:
with open(hrefs_out + 'hrefs_0', 'wb') as fp:
    pickle.dump(hrefs_0, fp)

In [ ]:
print('Number of edges in the starting layer is {0}.'.format(len(hrefs_0)))

### First layer

In [ ]:
hrefs_1 = [sort_wiki_hrefs(url=url) for url in hrefs_0]

In [ ]:
with open(hrefs_out + 'hrefs_1', 'wb') as fp:
    pickle.dump(hrefs_1, fp)

In [ ]:
count = 0
for n in hrefs_1:
    count += len(n)

print('Number of edges after the first layer is {0}.'.format(count))

As we can see, even after the first layer, there are 53284 differnt edges in the graph if we start on a Wikipedia page, which makes this problem very hard to solve. For a good visualization we need at least 4-5 layers of references, so I have to choose another starting URL.

### b) ELTE webpage -- https://www.elte.hu/

For the second attempt I've tried using https://www.elte.hu/ as my starting URL.

In [ ]:
elte_start = 'https://www.elte.hu/'
hrefs_out = './hrefs/elte_hrefs/'

In [ ]:
def sort_elte_hrefs(url):
    
    html_page = urllib.request.urlopen(url)
    soup = BeautifulSoup(html_page, 'html.parser')

    hrefs = []
    for link in soup.findAll('a'):
        tmp_link = link.get('href')
        if tmp_link is not None:
            print(tmp_link)
            print(tmp_link[0])
            if tmp_link[0] != '#':
                if 'http' not in tmp_link:
                    hrefs.append(elte_start + tmp_link)
                else:
                    hrefs.append(tmp_link)

    # Sort out references which are listed multiple times
    return list(np.unique(hrefs))

### Starting URL

In [ ]:
start_url = 'https://www.elte.hu/'

In [ ]:
hrefs_0 = sort_elte_hrefs(url=start_url)

In [ ]:
with open(hrefs_out + 'hrefs_0', 'wb') as fp:
    pickle.dump(hrefs_0, fp)

In [ ]:
print('Number of edges in the starting layer is {0}.'.format(len(hrefs_0)))

In [ ]:
sort_elte_hrefs(url='https://www.elte.hu/allaspalyazatok')

In [ ]:
hrefs_0

### First layer

In [ ]:
hrefs_1 = [sort_elte_hrefs(url=url) for url in hrefs_0]

In [ ]:
with open(hrefs_out + 'hrefs_1', 'wb') as fp:
    pickle.dump(hrefs_1, fp)

In [ ]:
count = 0
for n in hrefs_1:
    count += len(n)

print('Number of edges after the first layer is {0}.'.format(count))